# PupilLabs Reference Image Mapper for AOIs in dynamic scenes

The biggest difference when using the Reference Image Mapper for dynamic scenes is that we will now need multiple reference images. Each reference image now represents one individual painting, so there is no more difference between the AOI and the reference images.

The gaze data is still stored in one folder per reference image - so we will end up having as many data folders as we have AOIs to analyse. While this sounds like we might have more information here that we had in the static case, we actually loose something important: the reference images don't reveal anything about their location in the art gallery. We won't know from looking at them how they were positioned relative to each other. 

Still, we can compute metrics that are comparable, or even equal to the metrics in the static case.

## Step 1: Load the data

Because we need to load the data from a bunch of different sub-folders, we create a function that loops through the files for us.

## Step 1: Load the data

Because we need to load the data from a bunch of different sub-folders, we create a function that loops through the files for us.

In [ ]:
dynamic_path = './reference_export/walking/'

def get_file_list(file_name, directory = dynamic_path):
    all_aoi_recordings = os.listdir(directory)
    return [f"{directory}{aoi}/{file_name}" for aoi in all_aoi_recordings]

def load_csv_files(file_name):
    
    if not '.csv' in file_name:
        file_name = file_name + '.csv'
        
    file_list = get_file_list(file_name)
    section_list = get_file_list('sections.csv')
    data = pd.DataFrame()
    
    for file, section in zip(file_list, section_list):
        
        df = pd.read_csv(file)
        sections = pd.read_csv(section)
        df['recording name'] = df['recording id'].replace(sections['recording id'].values, sections['recording name'].values)
        df['AOI'] = file.split('MAPPER_')[1].split('_walking')[0]
        
        data = pd.concat([data,df], ignore_index = True)
        
    return data

dynamic_sections = load_csv_files('sections.csv')
dynamic_gaze = load_csv_files('gaze.csv')
dynamic_fixations = load_csv_files('fixations.csv')
dynamic_fixations['AOI'] = dynamic_fixations['AOI'].replace({
    'Adel_Dauood_2': 'AD 2',
    'Adel_Dauood_3': 'AD 3',
    'Adel_Dauood_4': 'AD 4',
    'Christopher_Blanc_2': 'Blanc 2',
    'Gary_Cain' : 'Cain',
    'Gunlief_Grube': 'Grube',
    'Ingerlise_Vikne': 'Vikne',
    'Vivian_Hoi_Nielsen': 'Nielsen'
    
})

dynamic_images = get_file_list('reference_image.jpeg')
# observer ids
dynamic_observers = np.unique(dynamic_gaze['recording name'])

#### Look at the reference images

In [ ]:
# create a plot for the images
fig_reference, ax_reference = plt.subplots(int(np.ceil(len(dynamic_images)/4)),4, figsize = (25,12))

for ax, im in zip(ax_reference.flatten(),dynamic_images):
    dynamic_reference_image = Image.open(im)
    ax.imshow(np.asarray(dynamic_reference_image))
    ax.set_title(im.split('MAPPER_')[1].split('_walking')[0])
    
plt.savefig('./images/multiple_reference_images.png')

You can see how each reference image shows one painting. That means, we can skip the steps in which we define the AOIs - they are defined by the identity of the reference image.

## Step 2: Filter the data for gaze/fixation on a reference image

The eye tracker keeps recording even when none of the paintings of interest is in sight. This results in many gaze and fixations instances that are not detected on an reference image, but somewhere in between. 

Before we want to analyse the data, we have to remove gaze samples outside of the reference imagess. The column 'gaze detected on refernence image' gives us a handy way to do so.

In [ ]:
print(f"The gaze data frame contained originally {len(dynamic_gaze)} samples.")
print(f"The fixation data frame contained originally {len(dynamic_fixations)} samples.")

# filter the data frames 
dynamic_gaze = dynamic_gaze[dynamic_gaze['gaze detected in reference image']] 
dynamic_fixations_unfiltered = dynamic_fixations.copy()
dynamic_fixations = dynamic_fixations[dynamic_fixations['fixation detected in reference image']] 

print(f"After cleaning, the gaze data frame contained {len(dynamic_gaze)} samples.")
print(f"After cleaning, the fixation data frame contained {len(dynamic_fixations)} samples.")


## Step 3: Visual Journey
Showing scanpaths between the images is not possible anymore. 

So instead of visualising how the eyes travel from image to image in space, we will show how they travel from image to image in time, using an approach we call **visual journey**. 
In the visual journey, we visualize the times when fixations were detected on any of the reference images. 

Since our participants all started their walk in the museum at different times, we will align the data to the start of the section.

In [ ]:
dynamic_fixations.head()

In [ ]:
def align_timestamps(df, time_column, observers = dynamic_observers):
    for observer in observers:
        obs_idx = df[df['recording name'] == observer].index
        section_start_time = min(dynamic_sections[dynamic_sections['recording name'] == observer]['section start time [ns]'].values)
        df.loc[obs_idx, 'aligned timestamp [s]'] = (df.loc[obs_idx, time_column] - section_start_time)/1e9 
        
    return df['aligned timestamp [s]']

dynamic_gaze['aligned timestamp [s]'] = align_timestamps(dynamic_gaze, 'timestamp [ns]')

# next, we can plot the visual journey
sns.catplot(
    data=dynamic_gaze,
    x="aligned timestamp [s]",
    y="recording name",
    hue="AOI",
    aspect=1,
    linewidth=0,
    s=1,
    palette = 'YlGnBu',
    height = 7
);

## Step 4: Metrics
We can compute some metrics on this data, too.

### Reach

In [ ]:
dynamic_fixation_counts = dynamic_fixations.pivot_table(index='recording name', 
                      columns = 'AOI',
                      values='fixation id',
                      fill_value=0, 
                      aggfunc='count').unstack().to_frame().rename(columns={0:'fixation count'})
dynamic_fixation_counts.reset_index(inplace = True)

# We can generalize this plot by showing what proportion of subjects had at least on fixation of this image
dynamic_hits = dynamic_fixation_counts.copy()
dynamic_hits.loc[:,'hit'] = dynamic_hits['fixation count']>0

dynamic_reach = dynamic_hits.groupby('AOI').mean().reset_index()
dynamic_reach.loc[:,'hit'] =  dynamic_reach['hit']*100

sns.catplot(x="AOI", y="hit", kind="bar", data=dynamic_reach, palette = "Greens")
plt.xticks(rotation = 90)


plt.subplots_adjust(bottom=0.21)
plt.savefig('./images/hit_rate_dynamic')

In [ ]:
Each of our defined AOIs was looked at by all observers in the study.

### Time to first contact
Computing the time to first contact is a bit unfair in this case. The rooms were visited in a specific order that forces a temporal pattern on the order in which the imaged are seen. 

In [ ]:
dynamic_fixations['aligned timestamp [s]'] = align_timestamps(dynamic_fixations, 'start timestamp [ns]')

dynamic_first_contact = pd.DataFrame(dynamic_fixations.groupby(['recording name', 'AOI']).min()['aligned timestamp [s]'])
dynamic_first_contact.reset_index(inplace = True)

In [ ]:
# Finally, we can compute the mean time to first contact
sns.catplot(x="AOI", y="aligned timestamp [s]", kind="bar", data=dynamic_first_contact, palette = "Greens", ci = 95);
plt.xticks(rotation = 90)

plt.subplots_adjust(bottom=0.21)
plt.savefig('./images/first_contact_dynamic')

### Dwell time

In [ ]:
# add recording name for better readablity
dynamic_dwell = dynamic_fixations.groupby(['recording name', 'AOI']).sum()['duration [ms]']
dynamic_dwell = pd.DataFrame(dynamic_dwell)
dynamic_dwell.reset_index(inplace = True)

In [ ]:
sns.catplot(x="AOI", y="duration [ms]", kind="bar", data=dynamic_dwell, palette = "Greens", ci = 95)

plt.xticks(rotation = 90)
plt.subplots_adjust(bottom=0.21)
plt.savefig('./images/dwell-time_dynamic.png');

In [ ]:
# Gaze on Blanc 2
blanc_fixations = dynamic_fixations[dynamic_fixations['AOI'] == 'Blanc 2']

In [ ]:
blanc_example_fixations = blanc_fixations[blanc_fixations['recording name']== 'JR_U_W']


blanc_fig, blanc_axs = plt.subplots(1,1, figsize = (10,7))
blanc_axs.imshow(np.asarray(dynamic_reference_image))
#blanc_axs.scatter(blanc_example_fixations['fixation x [px]'],
#                     blanc_example_fixations['fixation y [px]'],facecolor='none', edgecolor='cyan', 
#                     linewidth=2, s = blanc_example_fixations['duration [ms]']);
#blanc_axs.plot(blanc_example_fixations['fixation x [px]'], blanc_example_fixations['fixation y [px]'], 
#                  color='grey');

plt.savefig('./blanc_clean.png');

Meeting Marc and Richard

- AOI size
- where


- who


- what
case study - research question (perception in a gallery)
vs areas of interest with the reference image mapper
content roadmap
pull out how-tos, methods from notebook

cloud tutorial session -> what's there now?
make your first enrichment
stuff on a shelf
supermarket was never released - linked to individual, but not 
what in this thing generalizes to a lot of stuff? 


- How-To steps 
csv data - enrichment 